In [15]:
from ultralytics import YOLO

model = YOLO("best_12_08_2025.pt")
# model = YOLO("best_20_08_2025_monika_agatka_e_100_batch_8.pt")

In [16]:
import cv2
import numpy as np

def contour_fit_percentage(contour):
    # Fit ellipse to contour
    ellipse = cv2.fitEllipse(contour)
    (center, axes, angle) = ellipse
    
    # Ellipse parameters
    cx, cy = center
    a = axes[0] / 2.0   # semi-major axis
    b = axes[1] / 2.0   # semi-minor axis
    theta = np.radians(angle)

    inside = 0
    total = len(contour)

    # Rotation matrix for ellipse alignment
    cos_t, sin_t = np.cos(theta), np.sin(theta)

    for pt in contour:
        x, y = pt[0]

        # Translate point relative to center
        xt = x - cx
        yt = y - cy

        # Rotate into ellipse-aligned coords
        xr = xt * cos_t + yt * sin_t
        yr = -xt * sin_t + yt * cos_t

        # Check ellipse equation
        if (xr**2) / (a**2) + (yr**2) / (b**2) <= 1:
            inside += 1

    return inside / total


In [17]:
import cv2
import numpy as np

def contour_thickness_along_major_axis(contour, draw=True):
    # Fit ellipse
    ellipse = cv2.fitEllipse(contour)
    (center, axes, angle) = ellipse
    cx, cy = center

    # Rotate contour so major axis is horizontal
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.transform(contour, M)

    # Bounding box in rotated coordinates
    x, y, w, h = cv2.boundingRect(rotated)

    thicknesses = []
    lines = []

    ys = rotated[:, 0, 1]
    y_min, y_max = ys.min(), ys.max()
    max_y_dim = y_max - y_min


    for xi in range(x, x + w):
        ys = rotated[(rotated[:, 0, 0] == xi)][:, 0, 1]
        if len(ys) > 1:
            y_min, y_max = ys.min(), ys.max()
            thickness = y_max - y_min
            thicknesses.append(thickness)
            if draw:
                lines.append(((xi, int(y_min)), (xi, int(y_max))))
                
    # print(f"test max dim: {np.max(thicknesses)} vs {max_y_dim}")
    thicknesses.append(max_y_dim)
    mean_thickness_px = np.mean(thicknesses)
    max_thickness_px = np.max(thicknesses)
    min_thickness_px = np.min(thicknesses)

    result = {
        "mean_lenght_px": mean_thickness_px,
        "max_lenght_px":  max_thickness_px,
        "min_lenght_px":  min_thickness_px
    }

    if draw:
        canvas = np.zeros((y + h + 20, x + w + 20, 3), dtype=np.uint8)

        # Draw the filled contour first (with a light gray color)
        cv2.drawContours(canvas, [rotated], -1, (100, 100, 100), cv2.FILLED)
        cv2.drawContours(canvas, [rotated], -1, (0, 255, 0), 1)
        for p1, p2 in lines:  # draw subset of lines
            cv2.line(canvas, p1, p2, (0, 0, 255), 1)
        result["canvas"] = canvas

    return result


In [18]:
def contour_thickness_along_minor_axis(contour, draw=True):
    # Fit ellipse
    ellipse = cv2.fitEllipse(contour)
    (center, axes, angle) = ellipse
    cx, cy = center

    # Rotate contour so major axis is horizontal
    M = cv2.getRotationMatrix2D(center, angle + 90, 1.0)
    rotated = cv2.transform(contour, M)

    # Bounding box in rotated coordinates
    x, y, w, h = cv2.boundingRect(rotated)

    thicknesses = []
    lines = []

    xs = np.round(rotated[:, 0, 0]).astype(int)
    unique_xs = np.unique(xs)

    for xi in unique_xs:
        ys = rotated[xs == xi][:, 0, 1]
        if len(ys) > 1:
            y_min, y_max = ys.min(), ys.max()
            thickness = y_max - y_min
            thicknesses.append(thickness)
            if draw:
                lines.append(((xi, int(y_min)), (xi, int(y_max))))

    # fallback: if no vertical spans found, force global
    if len(thicknesses) == 0:
        ys = rotated[:, 0, 1]
        xs = rotated[:, 0, 0]
        xi_mid = int(np.mean(xs))
        y_min, y_max = ys.min(), ys.max()
        thickness = y_max - y_min
        thicknesses.append(thickness)
        if draw:
            lines.append(((xi_mid, int(y_min)), (xi_mid, int(y_max))))

    ys = rotated[:, 0, 1]
    y_min, y_max = ys.min(), ys.max()
    max_y_dim = y_max - y_min
    if len(thicknesses) == 0:
        thicknesses.append(max_y_dim)

    print(f"test max dim: {np.max(thicknesses)} vs {max_y_dim}")
    # thicknesses.append((max_y_dim+np.max(thicknesses))/2)
    # thicknesses.append(max_y_dim)
    mean_thickness_px = np.mean(thicknesses)
    max_thickness_px = np.max(thicknesses)
    min_thickness_px = np.min(thicknesses)

    result = {
        "mean_thickness_px": mean_thickness_px,
        "max_thickness_px": max_thickness_px,
        "min_thickness_px": min_thickness_px
    }

    if draw:
        canvas = np.zeros((y + h + 20, x + w + 20, 3), dtype=np.uint8)

        # Draw the filled contour first (with a light gray color)
        cv2.drawContours(canvas, [rotated], -1, (100, 100, 100), cv2.FILLED)

        # Draw the contour outline on top
        cv2.drawContours(canvas, [rotated], -1, (0, 255, 0), 1)

        for p1, p2 in lines:  # draw subset of lines
            cv2.line(canvas, p1, p2, (0, 0, 255), 1)
        result["canvas"] = canvas

    return result

In [19]:
import cv2
import numpy as np
import os

def analyze_bacterium_from_image(original_image_path, scale_factor_mm_per_pixel, total_image_area_mm2,x_pos=0,y_pos=0,bacteria_index=0):
    """
    Analyzes a single cropped image of a bacterium by:
    1. Creating a binary mask using Otsu's thresholding.
    2. Finding contours of the bacterium from the mask.
    3. Calculating all requested metrics in both pixels and millimeters.

    Args:
        original_image_path (str): Path to the cropped image file.
        scale_factor_mm_per_pixel (float): Conversion factor from pixels to millimeters.
        total_image_area_mm2 (float): The total area of the original image in mm^2.

    Returns:
        list of dict: A list of dictionaries with all calculated metrics.
    """
    try:
        # Load the original image
        original_img = cv2.imread(original_image_path)
        if original_img is None:
            print(f"Error: Could not read the image '{original_image_path}'.")
            return []

    except Exception as e:
        print(f"An error occurred while loading the image: {e}")
        return []

    # Convert the image to grayscale and create the mask
    gray_image = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.dilate(mask, kernel, iterations=1)


    # Find contours in the mask (each contour represents a detected object)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    results = []
    object_count = len(contours)
    total_masked_area_px = cv2.countNonZero(mask)

    largest_contour = max(
        [c for c in contours if cv2.contourArea(c) >= 0],
        key=cv2.contourArea
    )
    contour = largest_contour
        
    M = cv2.moments(contour)

    # 1. Pixel Coordinates
    x_pix = x_pos
    y_pix = y_pos
    
    # 2. Real-World Coordinates (mm)
    x_mm = x_pix * scale_factor_mm_per_pixel
    y_mm = y_pix * scale_factor_mm_per_pixel

    # 3. Area, Perimeter & Ellipse Metrics
    contour_area_px = cv2.contourArea(contour)
    contour_area_mm2 = contour_area_px * (scale_factor_mm_per_pixel ** 2)
    
    perimeter_px = cv2.arcLength(contour, True)
    perimeter_mm = perimeter_px * scale_factor_mm_per_pixel

    # Fit an ellipse to the contour
    try:
        contour_fit_in_ellipse = contour_fit_percentage(contour=contour)
        print(f"Countor in elipsa fit: {contour_fit_percentage(contour=contour)}")
        ellipse = cv2.fitEllipse(contour)
        (center, axes, angle) = ellipse
        # length_mm = max(axes) * scale_factor_mm_per_pixel

        contour_thickness_along_minor_axis_params = contour_thickness_along_minor_axis(contour=contour)
        contour_thickness_along_major_axis_params = contour_thickness_along_major_axis(contour=contour)

        # cv2.imshow("Contour Thickness", contour_thickness_along_major_axis_params["canvas"])
        # cv2.waitKey(0)

        # cv2.imshow("Contour Thickness", contour_thickness_along_minor_axis_params["canvas"])
        # cv2.waitKey(0)
        length_mm = contour_thickness_along_major_axis_params['max_lenght_px'] * scale_factor_mm_per_pixel

        # width_mm = min(axes) * scale_factor_mm_per_pixel * np.sqrt(np.sqrt(contour_fit_in_ellipse))
        width_mm = contour_thickness_along_minor_axis_params['max_thickness_px'] * scale_factor_mm_per_pixel
        
        # Perimeter of the fitted ellipse (Ramanujan's approximation)
        h = ((length_mm - width_mm) / (length_mm + width_mm)) ** 2
        perimeter_ellipse_mm = np.pi * (length_mm + width_mm) * (1 + 3 * h / (10 + np.sqrt(4 - 3 * h)))
    except:
        # Fallback for contours that are too small or not ellipse-like
        length_mm, width_mm, angle = 0, 0, 0
        perimeter_ellipse_mm = perimeter_mm

    # 4. Feret's and Sieve Diameter
    rect = cv2.minAreaRect(contour)
    width_rect_px = rect[1][0]
    height_rect_px = rect[1][1]
    
    feret_h_mm = max(width_rect_px, height_rect_px) * scale_factor_mm_per_pixel
    feret_v_mm = min(width_rect_px, height_rect_px) * scale_factor_mm_per_pixel
    
    # Martin's Diameter (using your simplification: major axis of the fitted ellipse)
    martin_diameter_mm = length_mm

    # Sieve Diameter: diameter of a circle with the same area
    sieve_diameter_mm = 2 * np.sqrt(contour_area_mm2 / np.pi)
    
    # 5. Color and Brightness Metrics
    object_mask = np.zeros(original_img.shape[:2], dtype="uint8")
    cv2.drawContours(object_mask, [contour], -1, 255, -1)
    
    object_pixels = original_img[object_mask == 255]
    
    b = object_pixels[:, 0]  # All rows, first column (blue)
    g = object_pixels[:, 1]  # All rows, second column (green)
    r = object_pixels[:, 2]  # All rows, third column (red)
    mean_b = np.mean(b) if b.size > 0 else 0
    mean_g = np.mean(g) if g.size > 0 else 0
    mean_r = np.mean(r) if r.size > 0 else 0
    
    mean_y = 0.299 * mean_r + 0.587 * mean_g + 0.114 * mean_b
    
    gray_original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)

    # Now, use the mask to extract only the pixels belonging to the bacterium from the grayscale image
    object_pixels_gray = gray_original_img[object_mask == 255]

    # Calculate the mean brightness of those extracted pixels
    mean_brightness = np.mean(object_pixels_gray) if object_pixels_gray.size > 0 else 0
    
    # 6. Total Image Metrics
    udzial_punktow = (total_masked_area_px * (scale_factor_mm_per_pixel ** 2) / total_image_area_mm2) * 100 if total_image_area_mm2 > 0 else 0

    results.append({
        'nr.': bacteria_index,
        'xpix.': x_pix,
        'ypix.': y_pix,
        'xmm': round(x_mm,1),
        'ymm': round(y_mm,1),
        'powierzchniamm': round(contour_area_mm2,2),
        'dlugoscmm': round(length_mm,8),
        'szerokoscmm': round(width_mm,8),
        'kat': round(angle,2),
        'obwodmm': round(perimeter_mm,8),
        'obwod_c.mm': round(perimeter_ellipse_mm,8),
        'srednica_fereta_hmm': round(feret_h_mm,1),
        'srednica_fereta_vmm': round(feret_v_mm,1),
        'sredn._martinamm': round(martin_diameter_mm,1),
        'sredn._sitowamm': round(sieve_diameter_mm,8),
        'srednia_jaskrawosc': round(mean_brightness,1),
        'r': round(mean_r,1),
        'g': round(mean_g,1),
        'b': round(mean_b,1),
        'y': round(mean_y,1),
        'liczenie_obiektow_w': int(object_count),
        'udzial_punktow': round(udzial_punktow,1),
        'pole_obrazu_mm2': round(total_image_area_mm2,1)
    })
        
    return results


In [20]:
def get_searching_results(img_path="test_image/a1.jpg",confidence_level=0.5,save_path=None,result_previev_form="id"):
    results = model(img_path, conf=confidence_level)

    if save_path is not None:
        if result_previev_form == "id":
            img = cv2.imread(img_path)

            for result in results:
                boxes = result.boxes.xyxy.cpu().numpy()  # x1, y1, x2, y2
                for idx, box in enumerate(boxes):
                    x1, y1, x2, y2 = map(int, box[:4])
                    
                    # Draw bounding box
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    # Write element ID
                    cv2.putText(img, f"ID:{idx+1}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            cv2.imwrite(save_path, img)
        else:
            for result in results:
                result.save(filename=save_path)  # save to disk


    return results

In [21]:
import pandas as pd

def convert_csv(input_df, output_txt,index_num=1,mode='w'):
    # Read CSV
    df = input_df

    # Extract top metrics from the first row
    # liczenie = int(df.loc[0, 'liczenie_obiektow_w'])
    # udzial = float(df.loc[0, 'udzial_punktow'])
    # pole = float(df.loc[0, 'pole_obrazu_mm2'])

    # Prepare header text
    header = (
        f"#{index_num}\n"
        "liczenie\nobiektów (w)\tudział punktów\tpole obrazu(mm²)\n"
        f"\t{len(df)}\t100.00\t12288.0000\n"
    )

    # Select only the object metrics
    cols = [
        'nr.', 'xpix.', 'ypix.', 'xmm', 'ymm', 'powierzchniamm',
        'dlugoscmm', 'szerokoscmm', 'kat', 'obwodmm', 'obwod_c.mm',
        'srednica_fereta_hmm', 'srednica_fereta_vmm', 'sredn._martinamm',
        'sredn._sitowamm', 'srednia_jaskrawosc', 'r', 'g', 'b', 'y'
    ]
    df_obj = df[cols].copy()

    # Format floats
    df_obj = df_obj.applymap(
        lambda x: f"{x:0.8f}" if isinstance(x, float) else x
    )

    # Column names mapping (your requested names)
    col_names = [
        "nr.", "x(pix.):", "y(pix.):", "x(mm):", "y(mm):", "powierzchnia:(mm²)",
        "długość:(mm)", "szerokość:(mm)", "kąt:(°)", "obwód:(mm)", "obwód C.:(mm)",
        "średnica Fereta H(mm)", "średnica Fereta V(mm)", "średn. Martina:(mm)",
        "średn. sitowa:(mm)", "średnia jaskrawość", "R", "G", "B", "Y"
    ]
    df_obj.columns = col_names

    # Save to text
    with open(output_txt, mode, encoding="utf-8") as f:
        f.write("\n")
        f.write(header)
        f.write("\t".join(col_names) + "\n")
        for _, row in df_obj.iterrows():
            f.write("\t".join(map(str, row.values)) + "\n")


In [22]:
import pandas as pd

def convert_csv_to_full_raport(input_dfs, output_txt):
    with open(output_txt, 'w', encoding="utf-8") as f:
        f.write("")
    for i,df in enumerate(input_dfs):
        convert_csv(df,output_txt,i+1,'a')


In [23]:
def get_results_df(full_image_path,results):
    image = cv2.imread(full_image_path)

    results_list = []

    if len(results[0]) == 0:
            results_list = {
                'nr.': [],
                'xpix.': [],
                'ypix.': [],
                'xmm': [],
                'ymm': [],
                'powierzchniamm': [],
                'dlugoscmm': [],
                'szerokoscmm': [],
                'kat': [],
                'obwodmm': [],
                'obwod_c.mm': [],
                'srednica_fereta_hmm': [],
                'srednica_fereta_vmm': [],
                'sredn._martinamm': [],
                'sredn._sitowamm': [],
                'srednia_jaskrawosc': [],
                'r': [],
                'g': [],
                'b': [],
                'y': [],
                'liczenie_obiektow_w': [],
                'udzial_punktow': [],
                'pole_obrazu_mm2': []
            }

    # Ensure results[0] contains bounding boxes in format [x1, y1, x2, y2]
    for i, box in enumerate(results[0]):
        for det in box.boxes:
            x1, y1, x2, y2 = det.xyxy.int().tolist()[0] # Convert coordinates to integers

            # Crop the region of interest
            cropped = image[y1:y2, x1:x2]

            # Save cropped image to temporary path
            cropped_path = f"temp_cropped_{i}.jpg"
            cv2.imwrite(cropped_path, cropped)

            # Analysis parameters
            scale_mm_per_px = 0.1
            total_image_area_mm2 = abs(x1-x2)*abs(y1-y2)*scale_mm_per_px*scale_mm_per_px

            # Run analysis on the cropped image
            analysis_results = analyze_bacterium_from_image(cropped_path, scale_mm_per_px, total_image_area_mm2,x_pos=(x1+x2)/2,y_pos=image.shape[0]-(y1+y2)/2,bacteria_index=i+1)


            if analysis_results:
                for result in analysis_results:
                    results_list.append(result)
            else:
                print(f"Analysis failed or no objects were detected for Box #{i}.")

            # Optionally remove the temporary cropped image
            os.remove(cropped_path)

    return pd.DataFrame(results_list)

In [24]:
import cv2
import pandas as pd
import numpy as np
import os

def get_results_no_model_df(full_image_path, bboxes_file_path):
    # Load image
    image = cv2.imread(full_image_path)
    H, W = image.shape[:2]

    results_list = []

    # Load YOLO-format bounding boxes from file
    with open(bboxes_file_path, "r") as f:
        lines = f.readlines()

    if len(lines) == 0:
        return pd.DataFrame({
            'nr.': [],
            'xpix.': [],
            'ypix.': [],
            'xmm': [],
            'ymm': [],
            'powierzchniamm': [],
            'dlugoscmm': [],
            'szerokoscmm': [],
            'kat': [],
            'obwodmm': [],
            'obwod_c.mm': [],
            'srednica_fereta_hmm': [],
            'srednica_fereta_vmm': [],
            'sredn._martinamm': [],
            'sredn._sitowamm': [],
            'srednia_jaskrawosc': [],
            'r': [],
            'g': [],
            'b': [],
            'y': [],
            'liczenie_obiektow_w': [],
            'udzial_punktow': [],
            'pole_obrazu_mm2': []
        })

    # Parse YOLO format boxes
    for i, line in enumerate(lines):
        cls, x_c, y_c, w, h = map(float, line.strip().split())
        
        # Convert normalized coordinates -> absolute pixel coordinates
        x_c, y_c, w, h = x_c * W, y_c * H, w * W, h * H
        x1 = int(x_c - w/2)
        x2 = int(x_c + w/2)
        y1 = int(y_c - h/2)
        y2 = int(y_c + h/2)

        # Crop ROI
        cropped = image[y1:y2, x1:x2]

        # Save temporarily
        cropped_path = f"temp_cropped_{i}.jpg"
        cv2.imwrite(cropped_path, cropped)

        # Scale calibration (mm/px) — adapt to your setup
        scale_mm_per_px = 0.1
        total_image_area_mm2 = abs(x1-x2) * abs(y1-y2) * (scale_mm_per_px**2)

        # Run bacterium analysis
        analysis_results = analyze_bacterium_from_image(
            cropped_path,
            scale_mm_per_px,
            total_image_area_mm2,
            x_pos=(x1+x2)/2,
            y_pos=image.shape[0] - (y1+y2)/2,
            bacteria_index=i+1
        )

        if analysis_results:
            for result in analysis_results:
                results_list.append(result)
        else:
            print(f"Analysis failed or no objects detected for Box #{i}.")

        os.remove(cropped_path)

    return pd.DataFrame(results_list)


In [25]:
# import cv2
# import os
# import pandas as pd

# source_image = "test_data/a1.jpg"
# results = get_searching_results(img_path=source_image, save_path="test_results/a1_marked.jpg")
# df = get_results_df(source_image,results)
# df.to_csv('test_results/result_raport.csv',index=False)
# convert_csv(input_df=df,output_txt="test_results/a1_report.txt")


In [26]:
import os
import re
def process_all_images_in_catalog(path='x',no_model_mode=False):

    if not os.path.exists(f"{path}/results"):
        os.mkdir(f"{path}/results")

    def natural_sort_key(s):
        return [int(c) if c.isdigit() else c.lower() for c in re.split('([0-9]+)', s)]
    items = os.listdir(path)


    image_files = []
    for item in items:
        if os.path.isfile(os.path.join(path, item)) and item.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image_files.append(item)
    
    image_files = sorted(image_files,key=natural_sort_key)

    result_dfs = []

    # image_files = ['a2.jpg']

    for image_path in image_files:
        full_image_path = f"{path}/{image_path}"
        print(image_path)
        if no_model_mode:
            df = get_results_no_model_df(full_image_path,bboxes_file_path=f"{path}/real_boxes/agatka_1_y_{image_path[:-4]}.txt")
            df.to_csv(f"{path}/results/{image_path[:-4]}_raport_no_model.csv",index=False)
        else:
            results = get_searching_results(img_path=full_image_path, save_path=f"{path}/results/{image_path}_marked.jpg")
            df = get_results_df(full_image_path,results)
            df.to_csv(f"{path}/results/{image_path[:-4]}_raport.csv",index=False)
        result_dfs.append(df)
    if no_model_mode:
        convert_csv_to_full_raport(result_dfs,output_txt=f"{path}/results/full_raport_no_model.txt")
    else:
        convert_csv_to_full_raport(result_dfs,output_txt=f"{path}/results/full_raport.txt")


In [27]:
# process_all_images_in_catalog(path="raport_generating_test/y",no_model_mode=False)
process_all_images_in_catalog(path="raport_generating_test/y",no_model_mode=True)

a1.jpg
Countor in elipsa fit: 0.6818181818181818
test max dim: 11 vs 11
Countor in elipsa fit: 0.1625
test max dim: 9 vs 34
Countor in elipsa fit: 0.5555555555555556
test max dim: 5 vs 5
Countor in elipsa fit: 0.5
test max dim: 7 vs 7
Countor in elipsa fit: 0.6470588235294118
test max dim: 10 vs 12
Countor in elipsa fit: 0.4375
test max dim: 9 vs 11
Countor in elipsa fit: 0.4166666666666667
test max dim: 6 vs 7
Countor in elipsa fit: 0.4
test max dim: 2 vs 7
Countor in elipsa fit: 0.5454545454545454
test max dim: 5 vs 8
Countor in elipsa fit: 0.6111111111111112
test max dim: 6 vs 9
Countor in elipsa fit: 0.5
test max dim: 7 vs 8
Countor in elipsa fit: 0.4
test max dim: 7 vs 8
Countor in elipsa fit: 0.5
test max dim: 4 vs 7
Countor in elipsa fit: 0.5454545454545454
test max dim: 6 vs 7
Countor in elipsa fit: 0.5833333333333334
test max dim: 7 vs 7
Countor in elipsa fit: 0.6363636363636364
test max dim: 9 vs 12
Countor in elipsa fit: 0.5
test max dim: 7 vs 7
Countor in elipsa fit: 0.5454

/tmp/ipykernel_17522/3646247499.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_obj = df_obj.applymap(
/tmp/ipykernel_17522/3646247499.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_obj = df_obj.applymap(
/tmp/ipykernel_17522/3646247499.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_obj = df_obj.applymap(
/tmp/ipykernel_17522/3646247499.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_obj = df_obj.applymap(
/tmp/ipykernel_17522/3646247499.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_obj = df_obj.applymap(
/tmp/ipykernel_17522/3646247499.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_obj = df_obj.applymap(
/tmp/ipykernel_17522/3646247499.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFram

In [28]:
# for letter in ['raport_generating_test/x','raport_generating_test/y','raport_generating_test/z']:
#     process_all_images_in_catalog(path=letter)